In [1]:
!pip install -q flask-ngrok
!pip install -q pyngrok

In [2]:
from pyngrok import ngrok

In [3]:
from google.colab import userdata
key = userdata.get('grok_token')

In [4]:
!ngrok authtoken {key}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import requests


In [6]:
ngrok.kill()

public_url = ngrok.connect(8080)
print(public_url)

NgrokTunnel: "https://fa37-34-59-64-239.ngrok-free.app" -> "http://localhost:8080"


In [7]:
!mkdir templates

In [8]:
%%writefile templates/base.html
<!DOCTYPE html>
<html>
<head>
    <title>LangAlpha</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">
    <style>
        body {
            font-family: sans-serif;
        }
        .navbar {
            background-color: #333;
            overflow: hidden;
        }
        .navbar a {
            float: left;
            display: block;
            color: white;
            text-align: center;
            padding: 14px 16px;
            text-decoration: none;
        }
        .navbar a:hover {
            background-color: #ddd;
            color: black;
        }
        .search-container {
            text-align: center;
            margin-top: 50px;
        }
        .search-container input[type="text"] {
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 5px;
            width: 300px;
        }
        .search-container button {
            padding: 10px 20px;
            background-color: #007bff;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
        }
    </style>
</head>
<body>
    <div class="navbar">
        <a href="/">Home</a>
        <a href="/about">About Us</a>
        <a href="/analysis">Analysis</a>
        <a href="/ai">AI Chat</a>
    </div>
    {% block content %}{% endblock %}
</body>
</html>

Writing templates/base.html


In [9]:
%%writefile templates/title.html
{% extends "base.html" %}
{% block content %}

<div class="container search-container">
    <h1>Welcome to LangAlpha</h1>
    <h5>Put a ticker for analysis</h5>
    <form action="/search">
        <input type="text" name="tic" placeholder="Enter Stock Ticker" required>
        <button type="submit">Search</button>
    </form>
</div>

{% endblock %}

Writing templates/title.html


In [10]:
%%writefile templates/ABOUT.html
{% extends "base.html" %}
{% block content %}

<div class="container">
  <div class="row">
    <div class="col-md-12 text-center" style="padding-top: 100px;">
      <h1>Thanks for using our website!</h1>
      <h5>We are students from NYU working on this project for a class. We hope you enjoy using it!</h5>
      <img class="center-block" src="{{ image_url }}" />
    </div>
  </div>
</div>

{% endblock %}

Writing templates/ABOUT.html


In [27]:
countries = [
    "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda",
    "Argentina", "Armenia", "Australia", "Austria", "Azerbaijan", "Bahamas", "Bahrain",
    "Bangladesh", "Barbados", "Belarus", "Belgium", "Belize", "Benin", "Bhutan",
    "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei", "Bulgaria",
    "Burkina Faso", "Burundi", "Cabo Verde", "Cambodia", "Cameroon", "Canada",
    "Central African Republic", "Chad", "Chile", "China", "Colombia", "Comoros",
    "Congo (Congo-Brazzaville)", "Costa Rica", "Croatia", "Cuba", "Cyprus",
    "Czech Republic", "Democratic Republic of the Congo", "Denmark", "Djibouti",
    "Dominica", "Dominican Republic", "Ecuador", "Egypt", "El Salvador",
    "Equatorial Guinea", "Eritrea", "Estonia", "Eswatini", "Ethiopia", "Fiji",
    "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece",
    "Grenada", "Guatemala", "Guinea", "Guinea-Bissau", "Guyana", "Haiti",
    "Honduras", "Hungary", "Iceland", "India", "Indonesia", "Iran", "Iraq",
    "Ireland", "Israel", "Italy", "Ivory Coast", "Jamaica", "Japan", "Jordan",
    "Kazakhstan", "Kenya", "Kiribati", "Kuwait", "Kyrgyzstan", "Laos", "Latvia",
    "Lebanon", "Lesotho", "Liberia", "Libya", "Liechtenstein", "Lithuania",
    "Luxembourg", "Madagascar", "Malawi", "Malaysia", "Maldives", "Mali", "Malta",
    "Marshall Islands", "Mauritania", "Mauritius", "Mexico", "Micronesia",
    "Moldova", "Monaco", "Mongolia", "Montenegro", "Morocco", "Mozambique",
    "Myanmar", "Namibia", "Nauru", "Nepal", "Netherlands", "New Zealand",
    "Nicaragua", "Niger", "Nigeria", "North Korea", "North Macedonia", "Norway",
    "Oman", "Pakistan", "Palau", "Palestine", "Panama", "Papua New Guinea",
    "Paraguay", "Peru", "Philippines", "Poland", "Portugal", "Qatar", "Romania",
    "Russia", "Rwanda", "Saint Kitts and Nevis", "Saint Lucia",
    "Saint Vincent and the Grenadines", "Samoa", "San Marino", "Sao Tome and Principe",
    "Saudi Arabia", "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore",
    "Slovakia", "Slovenia", "Solomon Islands", "Somalia", "South Africa",
    "South Korea", "South Sudan", "Spain", "Sri Lanka", "Sudan", "Suriname",
    "Sweden", "Switzerland", "Syria", "Taiwan", "Tajikistan", "Tanzania", "Thailand",
    "Timor-Leste", "Togo", "Tonga", "Trinidad and Tobago", "Tunisia", "Turkey",
    "Turkmenistan", "Tuvalu", "Uganda", "Ukraine", "United Arab Emirates",
    "United Kingdom", "United States", "Uruguay", "Uzbekistan", "Vanuatu",
    "Vatican City", "Venezuela", "Vietnam", "Yemen", "Zambia", "Zimbabwe"
]

In [28]:
industry_us = [
    "Aerospace & Defense",
    "Agriculture",
    "Apparel & Accessories",
    "Automotive",
    "Beverage (Alcoholic)",
    "Beverage (Soft)",
    "Biotechnology",
    "Chemicals",
    "Construction & Engineering",
    "Consumer Electronics",
    "Consumer Goods",
    "Education Services",
    "Energy (Oil & Gas)",
    "Energy (Renewable)",
    "Entertainment & Media",
    "Environmental Services",
    "Financial Services (Banking)",
    "Financial Services (Insurance)",
    "Financial Services (Investment)",
    "Food Products",
    "Healthcare (Devices & Supplies)",
    "Healthcare (Providers & Services)",
    "Healthcare (Pharmaceuticals)",
    "Hospitality & Leisure",
    "Household Products",
    "Information Technology (Hardware)",
    "Information Technology (Services)",
    "Information Technology (Software)",
    "Internet & E-Commerce",
    "Logistics & Transportation",
    "Manufacturing (Advanced)",
    "Metals & Mining",
    "Personal Care & Beauty",
    "Professional Services",
    "Real Estate",
    "Retail (Online)",
    "Retail (Physical)",
    "Telecommunications",
    "Tobacco",
    "Utilities",
    "Waste Management"
]

In [30]:
industry_global = [
    "Agriculture",
    "Apparel & Textiles",
    "Automotive",
    "Beverage (Alcoholic)",
    "Beverage (Soft)",
    "Chemicals",
    "Construction & Infrastructure",
    "Consumer Goods",
    "Education Services",
    "Energy (Oil & Gas)",
    "Energy (Renewable)",
    "Entertainment & Media",
    "Financial Services",
    "Food & Beverage",
    "Healthcare",
    "Hospitality & Tourism",
    "Household Products",
    "Information Technology",
    "Internet & E-Commerce",
    "Logistics & Shipping",
    "Manufacturing (General)",
    "Metals & Mining",
    "Paper & Forest Products",
    "Personal Care",
    "Pharmaceuticals",
    "Professional Services",
    "Real Estate",
    "Retail",
    "Telecommunications",
    "Tobacco",
    "Travel & Transportation",
    "Utilities",
    "Waste & Recycling"
]

In [31]:
#need to fix this, required is not save
#not all are number too
%%writefile templates/ANALYSIS.html
{% extends "base.html" %}
{% block content %}

<div class="container">
    <div class="row">
        <div class="col-md-12" style="padding-top: 10px;">
            <h1>Stock Analysis</h1>
            <form action="/analysis" method="POST">
                <div class="form-group">
                    <label for="tic">Enter Stock Ticker:</label>
                    <input type="text" class="form-control" id="tic" name="tic" placeholder="e.g., AAPL" required>
                </div>
                <div class="form-group">
                    <label for="country">Country of incorporation:</label>
                    <select class="form-control" id="country" name="country" required>
                        <option value="">Select Country</option>
                        {% for country in countries %}
                            <option value="{{ country }}">{{ country }}</option>
                        {% endfor %}
                    </select>
                </div>
                <div class="form-group">
                    <label for="Industry (US)">Industry (US):</label>
                    <select class="form-control" id="Industry (US)" name="Industry (US)" required>
                        <option value="">Select Industry</option>
                        {% for industry in industry_us %}
                            <option value="{{ industry }}">{{ industry }}</option>
                        {% endfor %}
                    </select>
                </div>
                <div class="form-group">
                    <label for="Industry (Global)">Industry (Global):</label>
                    <select class="form-control" id="Industry (Global)" name="Industry (Global)" required>
                        <option value="">Select Industry</option>
                        {% for industry in industry_global %}
                            <option value="{{ industry }}">{{ industry }}</option>
                        {% endfor %}
                    </select>
                </div>
                <div class="form-group">
                    <label for="Revenue Growth">Revenue growth rate for next year:5:</label>
                    <input type="number" class="form-control" id="Revenue Growth" name="Revenue Growth" placeholder="Enter a growth rate" required>
                </div>
                <div class="form-group">
                    <label for="Operating Margin">Operating margin for next year:</label>
                    <input type="number" class="form-control" id="Operating Margin" name="Operating Margin" placeholder="Enter the margin rate" required>
                </div>
                <div class="form-group">
                    <label for="Compunded Growth">Compounded annual growth rate:</label>
                    <input type="number" class="form-control" id="Compunded Growth" name="Compunded Growth" placeholder="Enter the compounded rate" required>
                </div>
                <div class="form-group">
                    <label for="Pre-tax Margin">Target pre-tax operating margin:</label>
                    <input type="number" class="form-control" id="Pre-tax Margin" name="Pre-tax Margin" placeholder="Enter the pre-tax margin rate " required>
                </div>
                <div class="form-group">
                    <label for="Convergence">Years of convergence for margin:</label>
                    <input type="number" class="form-control" id="Convergence" name="Convergence" placeholder="Enter the convergence year" required>
                </div>
                <div class="form-group">
                    <label for="Short-term Capital Ratio">Sales to capital ratio (for years 1-5):</label>
                    <input type="number" class="form-control" id="Short-term Capital Ratio" name="Short-term Capital Ratio" placeholder="Enter the capital ratio" required>
                </div>
                <div class="form-group">
                    <label for="Long-term Capital Ratio">Sales to capital ratio (for years 6-10):</label>
                    <input type="number" class="form-control" id="Long-term Capital Ratio" name="Long-term Capital Ratio" placeholder="Enter the capital ratio" required>
                </div>
                <button type="submit" class="btn btn-primary">Submit</button>
            </form>
        </div>
    </div>
</div>

{% endblock %}

Overwriting templates/ANALYSIS.html


In [15]:
!mkdir static
!gdown --fuzzy https://drive.google.com/file/d/1t6ADxRgGGzqkwkNh2i2z9n3geb5FLSMi/view?usp=sharing
!mv new-york-university.jpg static

Downloading...
From: https://drive.google.com/uc?id=1t6ADxRgGGzqkwkNh2i2z9n3geb5FLSMi
To: /content/new-york-university.jpg
100% 7.66k/7.66k [00:00<00:00, 20.6MB/s]


In [16]:
import base64

# Load the image (replace 'static/your_image.png' with the actual path)
with open("static/new-york-university.jpg", "rb") as img_file:
    encoded_string = base64.b64encode(img_file.read()).decode('utf-8')

In [17]:
%%writefile templates/Chat.html
{% extends "base.html" %}
{% block content %}
<!DOCTYPE html>
<html>
<head>
  <title>AI Chat</title>
  <style>
    body {
      font-family: sans-serif;
      margin: 0; /* Remove default body margins */
    }

    /* Center the chat container */
    .chat-container {
      width: 80%;
      height: 80vh;
      display: flex;
      flex-direction: column;
      border: 1px solid #ccc; /* Add a border for visual distinction */
      padding: 20px; /* Add some padding within the container */
      position: absolute; /* Position absolutely within the body */
      top: 50%;
      left: 50%;
      transform: translate(-50%, -50%); /* Center using translate */
    }

    /* Style for chat messages */
    .chat-messages {
      flex: 1; /* Allow chat messages to take up available space */
      overflow-y: auto; /* Add vertical scrollbar if needed */
      padding: 10px;
    }

    /* Style for individual messages */
    .chat-messages div {
      margin-bottom: 10px;
    }

    /* Style for user messages */
    .chat-messages div:nth-child(odd) { /* User messages are odd-numbered */
      text-align: right;
    }

    /* Style for AI messages */
    .chat-messages div:nth-child(even) { /* AI messages are even-numbered */
      text-align: left;
    }

    /* Style for chat input area */
    .chat-input {
      display: flex;
      padding: 10px;
    }

    /* Style for input field */
    .chat-input input[type="text"] {
      flex: 1; /* Allow input field to take up available space */
      padding: 10px;
      border: 1px solid #ccc;
      border-radius: 5px;
      margin-right: 10px;
    }

    /* Style for send button */
    .chat-input button {
      padding: 10px 20px;
      background-color: #007bff;
      color: white;
      border: none;
      border-radius: 5px;
      cursor: pointer;
    }
  </style>
</head>
<body>
  <div class="chat-container">
    <div class="chat-messages" id="chat-messages">
      </div>
    <div class="chat-input">
      <input type="text" id="user-input" placeholder="Type your message..." onkeyup="handleKeyUp(event)">
      <button onclick="sendMessage()">Send</button>
    </div>
  </div>

  <script>
    function sendMessage() {
      var userInput = document.getElementById("user-input").value;
      var chatMessages = document.getElementById("chat-messages");

      if (userInput.trim() !== "") { // Check if input is not empty
        // Create a new message element for the user's message
        var userMessageElement = document.createElement("div");
        userMessageElement.textContent = "You: " + userInput;
        chatMessages.appendChild(userMessageElement);

        // Simulate a response (replace with actual AI response later)
        var response = "AI: This is a placeholder response.";
        var aiMessageElement = document.createElement("div");
        aiMessageElement.textContent = response;
        chatMessages.appendChild(aiMessageElement);

        // Clear the input field
        document.getElementById("user-input").value = "";

        // Scroll to the bottom of the chat messages
        chatMessages.scrollTop = chatMessages.scrollHeight;
      }
    }

    function handleKeyUp(event) {
      if (event.keyCode === 13) { // Check if Enter key is pressed
        sendMessage();
      }
    }
  </script>
</body>
</html>
{% endblock %}

Writing templates/Chat.html


In [18]:
#analysis code

#_____________

In [35]:
#need to be fixed/add the real analysis

%%writefile templates/results.html
{% extends "base.html" %}
{% block content %}
<div class="container">
    <h1>Stock Analysis Results</h1>

    <!-- So far I am only listing stuff, but we should change it to be the analysis -->
    <h2>Input Data</h2>
    <ul>
        <li><strong>Ticker:</strong> {{ tic }}</li>
        <li><strong>Country:</strong> {{ country }}</li>
        <li><strong>Industry (US):</strong> {{ industry_us }}</li>
        <li><strong>Industry (Global):</strong> {{ industry_global }}</li>
        <li><strong>Revenue Growth:</strong> {{ revenue_growth }}</li>
        <li><strong>Operating Margin:</strong> {{ operating_margin }}</li>
        <li><strong>Compounded Growth:</strong> {{ compounded_growth }}</li>
        <li><strong>Pre-tax Margin:</strong> {{ pre_tax_margin }}</li>
        <li><strong>Convergence Years:</strong> {{ convergence_years }}</li>
        <li><strong>Short-term Capital Ratio:</strong> {{ short_term_capital_ratio }}</li>
        <li><strong>Long-term Capital Ratio:</strong> {{ long_term_capital_ratio }}</li>
    </ul>

    <h2>Analysis Results</h2>
    <p>{{ results }}</p> <!-- change the variable to whatever the contains the results -->
</div>

{% endblock %}

Overwriting templates/results.html


In [34]:
from pyngrok import ngrok
# Setup a tunnel to the flask port 5000
ngrok.kill()

public_url = ngrok.connect(5000)
print(public_url)

from flask import Flask, render_template, request, url_for
from flask_ngrok import run_with_ngrok
import requests

app = Flask(__name__,
            template_folder = '/content/templates',
            static_folder='/content/static')
#run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/")
def home():
    return render_template("title.html")

@app.route("/about")
def About():
  image_url = url_for('static', filename='new-york-university.jpg')
  return render_template("ABOUT.html", image_url=image_url)

@app.route("/analysis", methods=['GET', 'POST'])
def analysis():
    """
    if request.method == 'POST':
        tic = request.form['tic']
        country = request.form['country']
        industry_us = request.form['Industry (US)']
        industry_global = request.form['Industry (Global)']
        revenue_growth = request.form['Revenue Growth']
        operating_margin = request.form['Operating Margin']
        compounded_growth = request.form['Compunded Growth']
        pre_tax_margin = request.form['Pre-tax Margin']
        convergence_years = request.form['Convergence']
        short_term_capital_ratio = request.form['Short-term Capital Ratio']
        long_term_capital_ratio = request.form['Long-term Capital Ratio']


        Add ANALYSIS code here:

        # Redirect to results page with form data and analysis results
        return render_template("results.html",
                               tic=tic,
                               country=country,
                               industry_us=industry_us,
                               industry_global=industry_global,
                               revenue_growth=revenue_growth,
                               operating_margin=operating_margin,
                               compounded_growth=compounded_growth,
                               pre_tax_margin=pre_tax_margin,
                               convergence_years=convergence_years,
                               short_term_capital_ratio=short_term_capital_ratio,
                               long_term_capital_ratio=long_term_capital_ratio,
                               results=analysis_results)
        """
    #else:
      # Render the analysis form if it's a GET request
    return render_template("ANALYSIS.html", countries=countries, industry_us=industry_us, industry_global=industry_global)


@app.route("/ai")
def Ai():
  return render_template("Chat.html")

app.run()

NgrokTunnel: "https://7cf5-34-59-64-239.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [30/Apr/2025 21:38:58] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Apr/2025 21:38:59] "GET /analysis HTTP/1.1" 200 -
